In [16]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from src.inference import get_feature_store

In [3]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1) ),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-23 15:02:17,678 INFO: Initializing external client
2025-03-23 15:02:17,678 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-23 15:02:18,473 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214685
Fetching data from 2025-02-22 19:02:17.677807+00:00 to 2025-03-23 18:02:17.677807+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.68s) 


In [4]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-22 20:00:00+00:00,2,0
1,2025-02-22 21:00:00+00:00,2,0
2,2025-02-22 22:00:00+00:00,2,0
3,2025-02-22 23:00:00+00:00,2,0
4,2025-02-23 00:00:00+00:00,2,0
...,...,...,...
176182,2025-03-23 14:00:00+00:00,263,125
176183,2025-03-23 15:00:00+00:00,263,90
176184,2025-03-23 16:00:00+00:00,263,99
176185,2025-03-23 17:00:00+00:00,263,106


In [5]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 176187 entries, 0 to 182282
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         176187 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  176187 non-null  int32                  
 2   rides               176187 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [6]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [7]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 176187 entries, 0 to 182282
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         176187 non-null  datetime64[us]
 1   pickup_location_id  176187 non-null  int32         
 2   rides               176187 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [8]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

Skipping location_id 99: Not enough data to create even one window.


In [9]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-23 15:02:27,764 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-23 15:02:27,767 INFO: Initializing external client
2025-03-23 15:02:27,767 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-23 15:02:28,472 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214685
Fetching data from 2025-02-22 19:02:27.764312+00:00 to 2025-03-23 18:02:27.764312+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.00s) 
Skipping location_id 99: Not enough data to create even one window.


In [10]:
current_date

Timestamp('2025-03-23 19:02:27.764312+0000', tz='Etc/UTC')

In [11]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-22 20:00:00
1,1,0,0,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,1,3,2025-03-22 20:00:00
2,15,17,26,56,84,117,119,66,18,2,...,18,30,20,17,16,11,10,10,4,2025-03-22 20:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,2025-03-22 20:00:00
4,0,0,0,0,0,0,0,0,0,1,...,0,0,1,1,0,0,0,0,6,2025-03-22 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-22 20:00:00
249,0,0,0,2,0,0,1,2,4,1,...,1,0,1,0,3,0,2,4,260,2025-03-22 20:00:00
250,25,16,20,25,24,11,6,1,4,3,...,36,31,49,63,51,64,51,39,261,2025-03-22 20:00:00
251,69,32,33,37,12,8,4,8,4,6,...,127,118,104,73,72,92,113,78,262,2025-03-22 20:00:00


In [12]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-23 15:02:37,997 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-23 15:02:37,999 INFO: Initializing external client
2025-03-23 15:02:38,000 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-23 15:02:38,658 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214685


In [13]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [14]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,15.0
3,5,0.0
4,6,0.0
...,...,...
248,259,0.0
249,260,2.0
250,261,35.0
251,262,59.0


In [15]:
predictions.sort_values("predicted_demand", ascending=False).head(10)

,pickup_location_id,predicted_demand
123,132,283.0
77,79,264.0
238,249,260.0
133,142,230.0
223,234,218.0
225,236,211.0
66,68,209.0
160,170,204.0
219,230,202.0
151,161,201.0
